In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
xl = pd.ExcelFile("./raw_data/NWM_data_updated_ST_less_2.5.xlsx", engine='openpyxl')
a=xl.sheet_names

                                                                                                                                
sheet_no = list(range(2, 32))

In [2]:
#loading the data set
dataframe_collection = {}
i=0
for i in range(33):
    if i==0 :
        dataframe_collection[i] = xl.parse(a[i],converters={"STAID_1":str})
    else:
        dataframe_collection[i] = xl.parse(a[i])


dataframe_collection[0]=dataframe_collection[0].dropna().reset_index(drop=True)

In [3]:
dataframe_collection[0]

,STAID_1,Dist2Sea(degree),LandUse,SoilTexture-1layer,SoilTexture-2layer,SoilTexture-3layer,SoilTexture-4layer,SoilTexture-5layer,SoilTexture-6layer,SoilTexture-7layer,SoilTexture-8layer,SoilTexture-9layer,SoilTexture-10layer,TopographyIndex,DrainageArea(sqml)
0,02223000,3.082207,3.0,2.0,2.0,2.0,2.0,12.0,12.0,7.0,7.0,7.0,15.0,7.739953,663.0
1,02223190,2.857361,3.0,2.0,2.0,2.0,2.0,2.0,12.0,7.0,7.0,7.0,15.0,7.918407,184.0
2,02223360,2.785046,3.0,2.0,2.0,2.0,2.0,2.0,7.0,7.0,7.0,7.0,15.0,7.938193,44.7
3,02315920,1.629911,2.0,2.0,2.0,2.0,2.0,2.0,2.0,7.0,7.0,7.0,15.0,8.697107,537.0
4,02316000,1.506092,2.0,2.0,2.0,2.0,2.0,2.0,2.0,7.0,7.0,7.0,15.0,8.778268,550.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,02355350,1.126060,2.0,2.0,2.0,2.0,3.0,2.0,7.0,7.0,7.0,7.0,7.0,9.204712,6930.0
73,02329342,0.668634,2.0,2.0,2.0,2.0,3.0,3.0,7.0,7.0,7.0,7.0,15.0,8.850727,121.0
74,02353265,1.453645,2.0,2.0,2.0,2.0,2.0,2.0,7.0,7.0,7.0,7.0,7.0,8.878219,181.0
75,02354350,1.491261,2.0,2.0,2.0,2.0,3.0,2.0,7.0,7.0,7.0,7.0,15.0,9.242209,207.0


In [4]:

x_mean = pd.read_csv("./raw_data/x_mean.csv", sep='\t')
x_std =  pd.read_csv("./raw_data/x_std.csv", sep='\t')
y_mean =  pd.read_csv("./raw_data/y_mean.csv", sep='\t')
y_std =  pd.read_csv("./raw_data/y_std.csv", sep='\t')

In [5]:
from keras.models import load_model
import statistics
import math
pred = [None] * 30
std = [None] * 30
std_err = [None] *30
obsflow = dataframe_collection[2]
for l in range(0,30) :
     
    model = load_model('./temporal_DLmodels/my_model_%d_day.h5'%(l))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    modelflow= dataframe_collection[l+3]
    temp =  [0] * 77
    for index, row in dataframe_collection[0].iterrows():
        column_names = ["dis2sea","landUse","soilTexture1", "soilTexture2","soilTexture3","soilTexture4","soilTexture5","soilTexture6","soilTexture7","soilTexture8","soilTexture9","soilTexture10","topoIndex","drainageArea","ModelFlow","Obsflow"]
        df = pd.DataFrame(columns = column_names)  
        k=0
        d=0
      
        sta_id = dataframe_collection[0].iloc[index,0]
        dis2sea=dataframe_collection[0].iloc[index,1]
        land_use=dataframe_collection[0].iloc[index,2]
        soil_texture1=dataframe_collection[0].iloc[index,3]
        soil_texture2=dataframe_collection[0].iloc[index,4]
        soil_texture3=dataframe_collection[0].iloc[index,5]
        soil_texture4=dataframe_collection[0].iloc[index,6]
        soil_texture5=dataframe_collection[0].iloc[index,7]
        soil_texture6=dataframe_collection[0].iloc[index,8]
        soil_texture7=dataframe_collection[0].iloc[index,9]
        soil_texture8=dataframe_collection[0].iloc[index,10]
        soil_texture9=dataframe_collection[0].iloc[index,11]
        soil_texture10=dataframe_collection[0].iloc[index,12]
        topoindex = dataframe_collection[0].iloc[index,13]
        drainagearea =  dataframe_collection[0].iloc[index,14]
        obs = obsflow[sta_id]/drainagearea
        mod = modelflow[sta_id]/drainagearea

        for i in range(481,979):
            df.loc[k]=[dis2sea]+[land_use]+[soil_texture1]+[soil_texture2]+[soil_texture3]+[soil_texture4]+[soil_texture5]+[soil_texture6]+[soil_texture7]+[soil_texture8]+[soil_texture9]+[soil_texture10]+[topoindex]+[drainagearea]+[mod[i]]+[obs[i]]
            k=k+1
        df =df.dropna()
            
        if df.empty:
            temp[index] = 0
            d=d+1
            print(index)
        else :
            x, y = df.iloc[ :, 0:15], df.iloc[:,15]
            x-= x_mean.iloc[l,1:16]
            x /= x_std.iloc[l,1:16]
            
            z=model.predict(x)
            
            x=x*x_std.iloc[l,1:16]
            x=x +x_mean.iloc[l,1:16]
            
            z-= y_mean.iloc[l,1]
            z /= y_std.iloc[l,1]
            
            z=z.ravel()
            z=abs(z)
            a=x.iloc[:,14]
            b=a-(1*z)
            c=a+(z*1)
            res=y.between(b,c)
            count = np.count_nonzero(res)
            temp[index] = count/len(y)   
       
    std[l]= statistics.stdev(temp)    
    pred[l] = sum(temp)/(77-d) 
    std_err[l] = (std[l]*2)/math.sqrt(77-d)
    del model


21
45
21
45
21
45
21
45
21
45
21
45
21
45
21
45
21
45
21
45
21
45
21
45
21
45
21
45
21
45
21
45
21
45
21
45
21
45
21
45
21
45
21
45
21
45
21
45
21
45
21
45
21
45
21
45
21
45
21
45


In [6]:
d_x= pd.DataFrame(pred)
d_y=pd.DataFrame(std)
d_z=pd.DataFrame(std_err)
d_xy=pd.concat([d_x, d_y, d_z], axis=1)
d_xy.columns = ['Pred', 'Std', 'Std_err']
d_xy
d_xy.to_csv("./graph_data/prediction_std_stderr_probability_less_2.5.csv")

In [7]:
d_xy

,Pred,Std,Std_err
0,0.887856,0.260873,0.059458
1,0.894675,0.262128,0.059744
2,0.884327,0.268928,0.061294
3,0.888339,0.262413,0.059809
4,0.878808,0.269383,0.061398
5,0.877594,0.269926,0.061522
6,0.877856,0.269054,0.061323
7,0.874157,0.271513,0.061884
8,0.870286,0.277152,0.063169
9,0.862018,0.283291,0.064568


In [11]:
pred

[0.8878560434825211,
 0.8946754212636295,
 0.8843265738698223,
 0.8883385802364462,
 0.87880796243431,
 0.877594457389095,
 0.8778555443851821,
 0.874156509921878,
 0.8702858236899402,
 0.8620175923867149,
 0.871535700375361,
 0.8637577112280106,
 0.8702296525951929,
 0.8552712641775483,
 0.8676899148093823,
 0.859553994846869,
 0.8636952077172017,
 0.8533615950323686,
 0.8475535055657415,
 0.8600156226583843,
 0.8675256959523454,
 0.8428022644464229,
 0.8610684647196309,
 0.8605521632391935,
 0.8752406209790807,
 0.8660683441484749,
 0.8555624233391493,
 0.8624947542802677,
 0.8461046573249918,
 0.8465567666367612]